##Group details

Utkarsh Jaiswal, 18EX20030, utkjaiswal58@gmail.com <br><br>

Saharsh Agarwal, 18NA3EP12,saharshagarwal8@gmail.com <br><br>

Tripatjot Singh, 18AG36022 , triptjot3@gmail.com <br><br>

Kanishka Halder, 17EC35012,kanishkahalder@gmail.com <br><br>

Aniruddha Chattopadhyay, 17QE30007, studyaniruddha@gmail.com <br><br>

Monjit Boro, 18EC10035 , david10boro@gmail.com <br><br>

Amarnath Reddy Mukkamalla, 17PH20022, 152amarnath@gmail.com <br><br>

In [ ]:
import os, findspark
env_vars = dict(os.environ.items())
findspark.init(env_vars['SPARK_HOME'])

import re
import numpy as np
import matplotlib.pyplot as plt
from pyspark import SparkContext

sc = SparkContext("local[*]", "spark_app") # running spark using context
sc.setLogLevel("Error")

rdd = sc.textFile('Wiki-Vote.txt')
rdd.take(5)

# No Header
rdd = rdd.zipWithIndex().filter(lambda key_value: key_value[1] > 3).keys()
rdd.take(3)

rdd = rdd.map(lambda line: line.split('\t')[:2])
rdd = rdd.map(lambda each: (int(each[0]), int(each[1]))).distinct().cache()
rdd.take(3)

rdd_in = rdd.map(lambda each: (each[1], each[0])).groupByKey()
rdd_out = rdd.groupByKey()
rdd = rdd_in.fullOuterJoin(rdd_out).cache()
rdd.take(3)

lookupDict = rdd.collectAsMap()
keys = sorted(lookupDict.keys())
n = len(keys)
print(n) # number of nodes

ha_score = dict(zip(lookupDict.keys(), n * [(1/n, 1/n)]))
print(ha_score[1000])

for i in range(100):
    ha_score1 = {}
    h_sum, a_sum = 0, 0
    
    for key in keys:
        h, a = 0, 0
        iter_in, iter_out = lookupDict[key]
        if iter_out is not None:
            for v in iter_out: h += ha_score[v][1]
        if iter_in is not None:
            for u in iter_in: a += ha_score[u][0]
        h_sum += h
        a_sum += a
        ha_score1[key] = (h, a)
    
    for key in keys:
        h1, a1 = ha_score1[key]
        ha_score[key] = (h1/h_sum, a1/a_sum)

print(ha_score[1000])

fout = open("scores.txt", "w")
for key in keys:
    h, a = ha_score[key]
    fout.write(f"{key}\t{h: .15f}\t{a: .15f}\n")
fout.close()